In [1]:
from glob import glob
from os import path
import seaborn as sns
import json
import pandas as pd
import os
import numpy as np
import re
from os.path import basename, splitext
import time

# from solver import Instance

from argparse import Namespace
import sys
sys.path.append('../solver')
# from solver_output import practice_print
from solver_rostering import run_roster_solver_objval_w_baseline

/home/clarice/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
list_city_dt = []

for file_in in os.listdir(r'../instances'):
    base_file = splitext(basename(file_in))[0]
    base_file_split = base_file.split('_')
    file = base_file_split[0]+'_'+base_file_split[1]
    list_city_dt.append(file)

list_city_dt = list(set(list_city_dt))
list_city_dt.sort()
print(list_city_dt)
print(len(list_city_dt))

['berlin_db=0.50', 'berlin_db=1.00', 'berlin_db=2.00', 'berlin_db=4.00', 'frankfurt_db=0.50', 'frankfurt_db=1.00', 'frankfurt_db=2.00', 'frankfurt_db=4.00', 'lyon_db=0.50', 'lyon_db=1.00', 'lyon_db=2.00', 'lyon_db=4.00', 'paris_db=0.50', 'paris_db=1.00', 'paris_db=2.00', 'paris_db=4.00']
16


In [ ]:
#Subset to berlin

list_city_dt_subset = []

for city_db in list_city_dt:
    if (city_db.find('berlin') > -1):
        list_city_dt_subset.append(city_db)

print(list_city_dt_subset)

In [4]:


RM = 1.5
GM = 0.8
h_min = 32
h_max = 48
max_n_diff = 3

for base_file in list_city_dt_subset:
# for base_file in ['lyon_db=0.50']:

    list_results = []
    dict_out = {'instance': [],
        'city': [],
        'demand_baseline': [],
        'type': [],
        'outsourcing_cost_multiplier': [],
        'region_multiplier': [],
        'global_multiplier': [],
        'model': [],
        'max_n_shifts': [],
        'workforce_size': [],
        'wage_costs': [],
        'objective_value': [],
        'objective_value_post_wage': []}

    #create inputs to run a trial workforce size
    weekday_ = f'{base_file}_dt=doublepeak.json'
    weekend_ = f'{base_file}_dt=uniform.json'

    instance_file_weekday = f"../instances/{weekday_}"
    instance_file_weekend = f"../instances/{weekend_}"

    shift_file_weekday = f"../shifts/{weekday_}"
    shift_file_weekend = f"../shifts/{weekend_}"

    expand_workforce_to_regions=True
    workforce_dict = {}

    #create dictionary for evaluating results by city/demand baseline
    results_out = f"../results/{base_file}.json"

    for OC in [1.2, 1.5, 1.8, 2.0, 2.5]:
    # for OC in [1.2]:
        for model in ['fixed', 'flex', 'partflex']:
        # for model in ['fixed']:
            if model == 'partflex':
                for max_n_shifts in range(2,5):
                    start_time = time.time()
                    dict_baseline, dict_results = run_roster_solver_objval_w_baseline(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff, max_n_shifts, expand_workforce_to_regions=True)
                    end_time = time.time()
                    run_time = (end_time - start_time).round(2)
                    dict_baseline['type'] = ['baseline']
                    dict_results['type'] = ['model']
                    df_ = pd.DataFrame(dict_baseline)
                    list_results.append(df_)
                    # list_results_all.append(df_)
                    df_ = pd.DataFrame(dict_results)
                    list_results.append(df_)
                    # list_results_all.append(df_)
                    for key in dict_out.keys():
                        dict_out[key].extend(dict_baseline[key])
                        dict_out[key].extend(dict_baseline[key])
                    print(f'Base: {base_file}, OC: {OC}, model: {model}-{max_n_shifts}, run_time: {run_time}')
            else:
                start_time = time.time()
                dict_baseline, dict_results = run_roster_solver_objval_w_baseline(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff, expand_workforce_to_regions=True)
                end_time = time.time()
                run_time = (end_time - start_time).round(2)
                dict_baseline['type'] = ['baseline']
                dict_results['type'] = ['model']
                df_ = pd.DataFrame(dict_baseline)
                list_results.append(df_)
                # list_results_all.append(df_)
                df_ = pd.DataFrame(dict_results)
                list_results.append(df_)
                # list_results_all.append(df_)
                for key in dict_out.keys():
                    dict_out[key].extend(dict_baseline[key])
                    dict_out[key].extend(dict_baseline[key])
                print(f'Base: {base_file}, OC: {OC}, model: {model}, run_time: {run_time}')
    #export optimal workforce size values
    with open(results_out, 'w') as f:
        json.dump(dict_out, f, indent=2)
    df_results = pd.concat(list_results, ignore_index = True)
    df_results.to_excel(f'../results/{base_file}.xlsx', index = False)



Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 104664 rows, 171528 columns and 1380599 nonzeros
Model fingerprint: 0x6391cf50
Variable types: 99120 continuous, 72408 integer (72408 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [3e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3686.0800723
Found heuristic solution: objective 3686.0799768
Presolve removed 98453 rows and 131621 columns
Presolve time: 0.53s
Presolved: 6211 rows, 39907 columns, 112837 nonzeros
Found heuristic solution: objective 3686.0800000
Variable types: 3442 continuous, 36465 integer (36465 binary)
Found heuristic solution: objective 2513.9400000

Root relaxation: objective 2.401060e+03, 246 iterat

In [ ]:
# list_results_all = []
# df_results_all = pd.concat(list_results_all, ignore_index = True)
# df_results_all.to_excel(f'../results/all_cities_and_demand_baselines.xlsx', index = False)
